In [1]:
!pip install kmodes
# pandas, numpy, sklearn


[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes

In [4]:
df = pd.read_csv("dataset.csv")

effort_clustering = KMeans(n_clusters=2, random_state=0).fit(df[['time_online', 'video_clicks']])
df['effort'] = effort_clustering.labels_

proactivity_clustering = KMeans(n_clusters=2, random_state=0).fit(df[['content_anticipation', 'delay_lectures']])
df['proactivity'] = proactivity_clustering.labels_

control_clustering = KMeans(n_clusters=2, random_state=0).fit(df[['fraction_spent']])
df['control'] = control_clustering.labels_

df.to_csv("dataset_clustered.csv", index=False)

In [5]:
k_modes = KModes(n_clusters=4)
clusters = k_modes.fit_predict(df[['effort', 'proactivity', 'control']])
df['profile'] = clusters
df.to_csv("dataset_clustered.csv", index=False)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

features = df[["time_online", "video_clicks", "content_anticipation", "delay_lectures", "fraction_spent"]].values
target = df['grade'].values

scaler = StandardScaler()
features = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train).unsqueeze(1)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test).unsqueeze(1)

class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)
        self.fc2 = nn.Linear(10, 8)
        self.fc3 = nn.Linear(8, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
input_size = features.shape[1]
model = Net(input_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for i in range(10000):
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [13]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    mse = F.mse_loss(y_pred, y_test)
    print(mse.item())

1.101899266242981
